In [1]:
import os
import sys
import re
import time
import codecs
import numpy as np
import pandas as pd
import multiprocessing
from multiprocessing import Pool
import gc

In [2]:
minimum_cluster_size=10 #clusters with fewer than this won't be re-calculated; use original pfam consensus instead
ncpus=45
max_seq_len=253
gapchar='j'
padchar='9'
basedir='/data/saturn/a/hlim/pfam_clustered_corpora/'

In [3]:
blosum62 = {
    ('W', 'F'): 1, ('L', 'R'): -2, ('S', 'P'): -1, ('V', 'T'): 0,
    ('Q', 'Q'): 5, ('N', 'A'): -2, ('Z', 'Y'): -2, ('W', 'R'): -3,
    ('Q', 'A'): -1, ('S', 'D'): 0, ('H', 'H'): 8, ('S', 'H'): -1,
    ('H', 'D'): -1, ('L', 'N'): -3, ('W', 'A'): -3, ('Y', 'M'): -1,
    ('G', 'R'): -2, ('Y', 'I'): -1, ('Y', 'E'): -2, ('B', 'Y'): -3,
    ('Y', 'A'): -2, ('V', 'D'): -3, ('B', 'S'): 0, ('Y', 'Y'): 7,
    ('G', 'N'): 0, ('E', 'C'): -4, ('Y', 'Q'): -1, ('Z', 'Z'): 4,
    ('V', 'A'): 0, ('C', 'C'): 9, ('M', 'R'): -1, ('V', 'E'): -2,
    ('T', 'N'): 0, ('P', 'P'): 7, ('V', 'I'): 3, ('V', 'S'): -2,
    ('Z', 'P'): -1, ('V', 'M'): 1, ('T', 'F'): -2, ('V', 'Q'): -2,
    ('K', 'K'): 5, ('P', 'D'): -1, ('I', 'H'): -3, ('I', 'D'): -3,
    ('T', 'R'): -1, ('P', 'L'): -3, ('K', 'G'): -2, ('M', 'N'): -2,
    ('P', 'H'): -2, ('F', 'Q'): -3, ('Z', 'G'): -2, ('X', 'L'): -1,
    ('T', 'M'): -1, ('Z', 'C'): -3, ('X', 'H'): -1, ('D', 'R'): -2,
    ('B', 'W'): -4, ('X', 'D'): -1, ('Z', 'K'): 1, ('F', 'A'): -2,
    ('Z', 'W'): -3, ('F', 'E'): -3, ('D', 'N'): 1, ('B', 'K'): 0,
    ('X', 'X'): -1, ('F', 'I'): 0, ('B', 'G'): -1, ('X', 'T'): 0,
    ('F', 'M'): 0, ('B', 'C'): -3, ('Z', 'I'): -3, ('Z', 'V'): -2,
    ('S', 'S'): 4, ('L', 'Q'): -2, ('W', 'E'): -3, ('Q', 'R'): 1,
    ('N', 'N'): 6, ('W', 'M'): -1, ('Q', 'C'): -3, ('W', 'I'): -3,
    ('S', 'C'): -1, ('L', 'A'): -1, ('S', 'G'): 0, ('L', 'E'): -3,
    ('W', 'Q'): -2, ('H', 'G'): -2, ('S', 'K'): 0, ('Q', 'N'): 0,
    ('N', 'R'): 0, ('H', 'C'): -3, ('Y', 'N'): -2, ('G', 'Q'): -2,
    ('Y', 'F'): 3, ('C', 'A'): 0, ('V', 'L'): 1, ('G', 'E'): -2,
    ('G', 'A'): 0, ('K', 'R'): 2, ('E', 'D'): 2, ('Y', 'R'): -2,
    ('M', 'Q'): 0, ('T', 'I'): -1, ('C', 'D'): -3, ('V', 'F'): -1,
    ('T', 'A'): 0, ('T', 'P'): -1, ('B', 'P'): -2, ('T', 'E'): -1,
    ('V', 'N'): -3, ('P', 'G'): -2, ('M', 'A'): -1, ('K', 'H'): -1,
    ('V', 'R'): -3, ('P', 'C'): -3, ('M', 'E'): -2, ('K', 'L'): -2,
    ('V', 'V'): 4, ('M', 'I'): 1, ('T', 'Q'): -1, ('I', 'G'): -4,
    ('P', 'K'): -1, ('M', 'M'): 5, ('K', 'D'): -1, ('I', 'C'): -1,
    ('Z', 'D'): 1, ('F', 'R'): -3, ('X', 'K'): -1, ('Q', 'D'): 0,
    ('X', 'G'): -1, ('Z', 'L'): -3, ('X', 'C'): -2, ('Z', 'H'): 0,
    ('B', 'L'): -4, ('B', 'H'): 0, ('F', 'F'): 6, ('X', 'W'): -2,
    ('B', 'D'): 4, ('D', 'A'): -2, ('S', 'L'): -2, ('X', 'S'): 0,
    ('F', 'N'): -3, ('S', 'R'): -1, ('W', 'D'): -4, ('V', 'Y'): -1,
    ('W', 'L'): -2, ('H', 'R'): 0, ('W', 'H'): -2, ('H', 'N'): 1,
    ('W', 'T'): -2, ('T', 'T'): 5, ('S', 'F'): -2, ('W', 'P'): -4,
    ('L', 'D'): -4, ('B', 'I'): -3, ('L', 'H'): -3, ('S', 'N'): 1,
    ('B', 'T'): -1, ('L', 'L'): 4, ('Y', 'K'): -2, ('E', 'Q'): 2,
    ('Y', 'G'): -3, ('Z', 'S'): 0, ('Y', 'C'): -2, ('G', 'D'): -1,
    ('B', 'V'): -3, ('E', 'A'): -1, ('Y', 'W'): 2, ('E', 'E'): 5,
    ('Y', 'S'): -2, ('C', 'N'): -3, ('V', 'C'): -1, ('T', 'H'): -2,
    ('P', 'R'): -2, ('V', 'G'): -3, ('T', 'L'): -1, ('V', 'K'): -2,
    ('K', 'Q'): 1, ('R', 'A'): -1, ('I', 'R'): -3, ('T', 'D'): -1,
    ('P', 'F'): -4, ('I', 'N'): -3, ('K', 'I'): -3, ('M', 'D'): -3,
    ('V', 'W'): -3, ('W', 'W'): 11, ('M', 'H'): -2, ('P', 'N'): -2,
    ('K', 'A'): -1, ('M', 'L'): 2, ('K', 'E'): 1, ('Z', 'E'): 4,
    ('X', 'N'): -1, ('Z', 'A'): -1, ('Z', 'M'): -1, ('X', 'F'): -1,
    ('K', 'C'): -3, ('B', 'Q'): 0, ('X', 'B'): -1, ('B', 'M'): -3,
    ('F', 'C'): -2, ('Z', 'Q'): 3, ('X', 'Z'): -1, ('F', 'G'): -3,
    ('B', 'E'): 1, ('X', 'V'): -1, ('F', 'K'): -3, ('B', 'A'): -2,
    ('X', 'R'): -1, ('D', 'D'): 6, ('W', 'G'): -2, ('Z', 'F'): -3,
    ('S', 'Q'): 0, ('W', 'C'): -2, ('W', 'K'): -3, ('H', 'Q'): 0,
    ('L', 'C'): -1, ('W', 'N'): -4, ('S', 'A'): 1, ('L', 'G'): -4,
    ('W', 'S'): -3, ('S', 'E'): 0, ('H', 'E'): 0, ('S', 'I'): -2,
    ('H', 'A'): -2, ('S', 'M'): -1, ('Y', 'L'): -1, ('Y', 'H'): 2,
    ('Y', 'D'): -3, ('E', 'R'): 0, ('X', 'P'): -2, ('G', 'G'): 6,
    ('G', 'C'): -3, ('E', 'N'): 0, ('Y', 'T'): -2, ('Y', 'P'): -3,
    ('T', 'K'): -1, ('A', 'A'): 4, ('P', 'Q'): -1, ('T', 'C'): -1,
    ('V', 'H'): -3, ('T', 'G'): -2, ('I', 'Q'): -3, ('Z', 'T'): -1,
    ('C', 'R'): -3, ('V', 'P'): -2, ('P', 'E'): -1, ('M', 'C'): -1,
    ('K', 'N'): 0, ('I', 'I'): 4, ('P', 'A'): -1, ('M', 'G'): -3,
    ('T', 'S'): 1, ('I', 'E'): -3, ('P', 'M'): -2, ('M', 'K'): -1,
    ('I', 'A'): -1, ('P', 'I'): -3, ('R', 'R'): 5, ('X', 'M'): -1,
    ('L', 'I'): 2, ('X', 'I'): -1, ('Z', 'B'): 1, ('X', 'E'): -1,
    ('Z', 'N'): 0, ('X', 'A'): 0, ('B', 'R'): -1, ('B', 'N'): 3,
    ('F', 'D'): -3, ('X', 'Y'): -1, ('Z', 'R'): 0, ('F', 'H'): -1,
    ('B', 'F'): -3, ('F', 'L'): 0, ('X', 'Q'): -1, ('B', 'B'): 4
}
#custom entries to the matrix
pairs=list(blosum62.keys())
singles=[]
for pair in pairs:
    blosum62[(pair[1],pair[0])]=blosum62[pair]
    singles.append(pair[0])
    singles.append(pair[1])
singles=list(set(singles))
for s in singles: #gap-substitution = -11
    blosum62[('-',s)]=-11
    blosum62[(s,'-')]=-11
    blosum62[('.',s)]=-11
    blosum62[(s,'.')]=-11
    blosum62[(s,'O')]=0
    blosum62[(s,'U')]=0
    blosum62[('O',s)]=0
    blosum62[('U',s)]=0
blosum62[('.','.')]=0
blosum62[('-','-')]=0
blosum62[('.','-')]=0
blosum62[('-','.')]=0
blosum62[('O','O')]=10
blosum62[('U','U')]=10
blosum62[('O','U')]=0
blosum62[('U','0')]=0
blosum62[('O','-')]=-11
blosum62[('U','-')]=-11
blosum62[('-','O')]=-11
blosum62[('-','U')]=-11
blosum62[('O','.')]=-11
blosum62[('U','.')]=-11
blosum62[('.','O')]=-11
blosum62[('.','U')]=-11

In [4]:
blosum62[('O','-')]

-11

In [5]:
cluster_dir=os.path.join(basedir,'pfam_cdhit_clusters906030/')
cluster_files=os.listdir(cluster_dir)
print(len(cluster_files))

17772


In [6]:
"""
How to collect MSA from Pfam
Parse each Pfam file and get sequence on the fly
"""
pfam_dir='/data/saturn/a/hlim/Pfam/Pfam/' #.aln as extension
#=GC seq_cons (consensus sequence from all family members; used in case sub-clusters are not sufficient)

In [7]:
def collect_pfam_msa(pfamid):
    pfam_msa={}
    pfam_accession={}
    with codecs.open(os.path.join(pfam_dir,pfamid+'.aln'),'r',encoding='utf-8', errors='replace') as inf:
        for line in inf:
            line=line.strip().split()
            if len(line)<2:
                continue
            if line[0].startswith("#"):
                if line[0]=="#=GS":
                    if line[2]=="AC":
                        #=GS U5QJ87_9CYAN/28-322        AC U5QJ87.1
                        seqid=line[1]
                        uni=line[3]
                        pfam_accession[seqid]=uni
                elif line[0]=="#=GC":
                    #=GC seq_cons
                    pfam_msa['consensus']=line[2]
            elif line[0].startswith('/'):
                break
            else:
                seqid=line[0]
                aligned_seq=line[1]
                pfam_msa[seqid]=aligned_seq
    return pfam_msa, pfam_accession
            

In [8]:
def select_positions_by_subcluster_consensus(seqlist,max_seq_len):
    #input: seqlist=['ac-de','a--de','accde','ad-de','accee'], max_seq_len=3
    #output: [0,3,4]
    consensus_scores=[0.0]*len(seqlist[0])
    for i,seq1 in enumerate(seqlist):
        for j,seq2 in enumerate(seqlist):
            if i<=j:
                continue
            try:
                for pos in range(len(seqlist[0])):
                    r1=seq1[pos].upper()
                    r2=seq2[pos].upper()
                    score=blosum62[(r1,r2)]
                    consensus_scores[pos]+=score
            except:
                continue
    selected_positions=sorted(np.argsort(consensus_scores)[::-1][:max_seq_len])
    return selected_positions

In [9]:
def select_positions_by_default_consensus(consensus,max_seq_len):
    hcpositions=[] #high-conserved
    lcpositions=[] #low-conserved
    cspositions=[] #conservative-substitution
    inspositions=[] #insertion '.'
    delpositions=[] #deletion '-'
    for i,aa in enumerate(consensus):
        if aa.isupper():
            hcpositions.append(i)
        elif aa.islower():
            lcpositions.append(i)
        elif aa=='+':
            cspositions.append(i)
        elif aa=='.':
            inspositions.append(i)
        elif aa=='-':
            delpositions.append(i)
        else:
            continue
    selected_positions=hcpositions+lcpositions+cspositions
    selected_positions=selected_positions[:max_seq_len]
    selected_positions=sorted(selected_positions)
    return selected_positions

In [10]:
pfam_computational_load={} #key: pfamid, value: total number of calculation
pfam_num_clusters={} #use to draw histogram
cluster_members=[] #use to draw histogram
for c in cluster_files:
    pfamid=c.replace('.clstr','')
    pfam_clusters={}
    pfam_msa, pfam_accession=collect_pfam_msa(pfamid)
    with open(os.path.join(cluster_dir,pfamid+'.clstr'),'r',encoding="utf-8") as inf:
        current_cluster='Cluster 0'
        for line in inf:
            if line.startswith('>'):
                current_cluster=line.replace('>','')
                pfam_clusters[current_cluster]=[]
                continue
            seqid=line.strip().split('>')[1].split('|')[0]
            pfam_clusters[current_cluster].append(seqid)
        seqlen=len(pfam_msa['consensus'])
        npairs=0
        for cl in pfam_clusters.keys():
            nc=len(pfam_clusters[cl])
            npairs += nc*(nc-1)/2
            cluster_members.append(nc)
        pfam_num_clusters[pfamid]=len(pfam_clusters)
        pfam_computational_load[pfamid]=npairs*seqlen


In [11]:
print(pfam_computational_load['PF00069.25'],pfam_computational_load['PF00001.21'])

4927442384622.0 82379799132.0


In [12]:
ordered_families = sorted(pfam_computational_load.items(), key=lambda kv: kv[1])
ordered_families[:10]

[('PF11504.8', 0),
 ('PF01003.19', 0),
 ('PF18357.1', 0),
 ('PF01525.16', 0),
 ('PF10954.8', 0),
 ('PF11621.8', 0),
 ('PF17444.2', 0),
 ('PF17085.5', 0),
 ('PF06392.11', 0),
 ('PF01543.17', 0)]

In [13]:
ordered_families[::-1][:5]

[('PF00005.27', 9319161838840.0),
 ('PF00171.22', 6744077346012.0),
 ('PF00069.25', 4927442384622.0),
 ('PF00012.20', 1546974556032.0),
 ('PF00083.24', 1458139329660.0)]

In [ ]:
"""
Prepare for clustered pfam representations
"""
def preprocess_pfam(x):
    pfamid=x
    pfam_clusters={} #key: cluster id, value: [seq ids]
    pfam_file=os.path.join(pfam_dir,pfamid+'.aln')
    pfam_msa, pfam_accession=collect_pfam_msa(pfamid)
    selected_positions_default=select_positions_by_default_consensus(pfam_msa['consensus'],max_seq_len)
    all_singlets=open(os.path.join(basedir,'all_pfam_singlet_withID/'+pfamid),'w')
    all_triplets=open(os.path.join(basedir,'all_pfam_triplet_withID/'+pfamid),'w')
    clustered_singlets=open(os.path.join(basedir,'pfam_clustered_singlets/'+pfamid),'w')
    clustered_triplets=open(os.path.join(basedir,'pfam_clustered_triplets/'+pfamid),'w')

    with open(os.path.join(cluster_dir,pfamid+'.clstr'),'r') as inf:
        current_cluster='Cluster 0'
        for line in inf:
            if line.startswith('>'):
                current_cluster=line.replace('>','')
                pfam_clusters[current_cluster]=[]
                continue
            seqid=line.strip().split('>')[1].split('|')[0]
            pfam_clusters[current_cluster].append(seqid)
              
    for cl in pfam_clusters.keys():
        if len(pfam_clusters[cl])<minimum_cluster_size:
            selected_positions=selected_positions_default
        else:
            seqlist=[pfam_msa[seqid] for seqid in pfam_clusters[cl]]
            try:
                selected_positions=select_positions_by_subcluster_consensus(seqlist,max_seq_len)
            except:
                continue
            
        for seqid in pfam_clusters[cl]:
            singlets=[]
            triplets=[]
            for i in selected_positions:
                first=pfam_msa[seqid][:i-1].replace('.','')
                second=pfam_msa[seqid][i]
                third=pfam_msa[seqid][i+1:].replace('.','')
                singlets.append(second.replace('.',padchar).replace('-',gapchar).lower())
                if len(first)>0:
                    first=first[-1]
                else:
                    first='.'
                if len(third)>0:
                    third=third[0]
                else:
                    third='.'
                triplet=''.join((first,second,third)).lower().replace('.',padchar).replace('-',gapchar)
                triplets.append(triplet)
            if len(singlets)<max_seq_len:
                for i in range(max_seq_len-len(singlets)):
                    singlets.append(gapchar)
            if len(triplets)<max_seq_len:
                for i in range(max_seq_len-len(triplets)):
                    triplets.append(gapchar*3)
            sentence_singlet=' '.join(singlets)
            sentence_triplet=' '.join(triplets)
            
            if seqid == pfam_clusters[cl][0]: #representative sequence for the cluster
                #it is used for pretraining. No sequence ID in the file needed
                clustered_singlets.write(sentence_singlet+"\n")
                clustered_triplets.write(sentence_triplet+"\n")
            all_singlets.write(seqid+'\t'+pfam_accession[seqid]+'\t'+sentence_singlet+'\n')
            all_triplets.write(seqid+'\t'+pfam_accession[seqid]+'\t'+sentence_triplet+'\n')
        
    all_singlets.close()
    all_triplets.close()
    clustered_singlets.close()
    clustered_triplets.close()
    gc.collect()
    
inputs=[]
for pfamid in ordered_families[17735:]:
    inputs.append(pfamid[0])

since=time.time()
with Pool(ncpus) as pool:
    num_families_processed=0
    for res in pool.imap_unordered(preprocess_pfam,inputs):
        num_families_processed+=1
elapsed=time.time()-since
print("{:.4f} minutes for {} families".format(elapsed/60.0, num_families_processed))

In [16]:
ordered_families[17750:17760]

[('PF00063.21', 540512438730.0),
 ('PF07992.14', 601605427480.0),
 ('PF00270.29', 616541012960.0),
 ('PF00873.19', 680584201440.0),
 ('PF00990.21', 684474117390.0),
 ('PF02786.17', 686029382790.0),
 ('PF00501.28', 734182934386.0),
 ('PF00563.20', 818297857558.0),
 ('PF07690.16', 849354366452.0),
 ('PF00072.24', 923627412825.0)]

In [ ]:
Pf00171.22